<a href="https://colab.research.google.com/github/himanshudas13/IPL2024-analysis/blob/main/Data_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup

In [4]:

url = "https://www.espncricinfo.com/series/indian-premier-league-2024-1410320/match-schedule-fixtures-and-results"
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")

divs = soup.find_all("div", class_="ds-p-4")

scorecards=[]
for div in divs:
  links = div.find_all('a', href=lambda href: href and 'scorecard' in href)
  for link in links:
    scorecards.append(link.get('href'))

scorecards = ['https://www.espncricinfo.com/' + s for s in scorecards]
print(scorecards)


['https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/chennai-super-kings-vs-royal-challengers-bengaluru-1st-match-1422119/full-scorecard', 'https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/punjab-kings-vs-delhi-capitals-2nd-match-1422120/full-scorecard', 'https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/kolkata-knight-riders-vs-sunrisers-hyderabad-3rd-match-1422121/full-scorecard', 'https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/rajasthan-royals-vs-lucknow-super-giants-4th-match-1422122/full-scorecard', 'https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/gujarat-titans-vs-mumbai-indians-5th-match-1422123/full-scorecard', 'https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/royal-challengers-bengaluru-vs-punjab-kings-6th-match-1422124/full-scorecard', 'https://www.espncricinfo.com//series/indian-premier-league-2024-1410320/chennai-super-kings-vs-gujarat-ti

In [5]:
def get_key(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  result_div = soup.find('div', class_='ds-text-tight-m ds-font-regular ds-text-typo-mid3')
# Extract and print the text content
  if result_div:
    match_details = result_div.text.strip()
    # Split the string by comma
    parts = match_details.split(',')
    # Join the first 3 parts (excluding " Indian Premier League")
    result = ','.join(parts[:3])
    return result


In [6]:
def get_winner(url):
  page = requests.get(url)
  str=get_key(url)
  soup = BeautifulSoup(page.content, "html.parser")
  result_texts = []
  for p in soup.find_all('p', class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo"):
    span = p.find('span')
    if span:
      result_texts.append([str, span.text.strip().split()[0]])
  return result_texts

In [7]:
def get_batting_scores(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  str=get_key(url)
  batting_divs = soup.find_all('div', class_="ds-p-0")
  all_batting_data = []

  for div in batting_divs:
    tables = div.find_all('table', class_='ds-w-full ds-table ds-table-md ds-table-auto ci-scorecard-table')
    for table in tables:
      rows = table.find_all('tr',class_="")
      for row in rows:
        # print("Classes for this row:", row.get('class'))
        cols = row.find_all('td')
        row_data = []
        for col in cols:
          #skip next to bat
          if col.get('class') == ['ds-font-bold', 'ds-bg-fill-content-alternate', 'ds-text-tight-m', 'ds-text-right', '!ds-font-regular'] or col.get('class') == []:
            continue
          #skip total score
          if col.get('class')==['ds-font-bold', 'ds-bg-fill-content-alternate', 'ds-text-tight-m', 'ds-min-w-max'] or col.get('class') == ['ds-font-bold', 'ds-bg-fill-content-alternate', 'ds-text-tight-m', 'ds-min-w-max', 'ds-text-right', 'ds-text-typo']:
            continue
          #skip overs and rr
          if col.get('class') ==['ds-font-bold', 'ds-bg-fill-content-alternate', 'ds-text-tight-m', 'ds-min-w-max', 'ds-flex', 'ds-items-center', '!ds-pl-[100px]']:
            continue
          row_data.append(col.text.strip())
          # print("Classes for this <td>:", col.get('class'), col.text.strip())

        all_batting_data.append([str,row_data])


  return all_batting_data



In [17]:

def get_bowling_scores(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.content, "html.parser")
  str=get_key(url)
  bowling_divs = soup.find_all('div', class_="ds-p-0")
  all_bowling_data = []

  for div in bowling_divs:
    tables = div.find_all('table', class_='ds-w-full ds-table ds-table-md ds-table-auto')
    for table in tables:
      rows = table.find_all('tr',class_="")
      for row in rows:
        # print("Classes for this row:", row.get('class'))
        cols = row.find_all('td')
        row_data = [col.text.strip() for col in cols]
        # row_data.append(str)
        all_bowling_data.append([str,row_data])

  return all_bowling_data


In [72]:
batting_data=[]
bowling_data=[]
winner=[]
for(url) in scorecards:
  batting_data.append(get_batting_scores(url))
  bowling_data.append(get_bowling_scores(url))
  winner.append(get_winner(url))


In [38]:
batting_df=pd.DataFrame(batting_data)
bowling_df=pd.DataFrame(bowling_data)
winner_df=pd.DataFrame(winner)
batting_df.head(20)
bowling_df.head(20)
# winner_df.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,"[1st Match (N), Chennai, March 22, []]","[1st Match (N), Chennai, March 22, [Deepak Cha...","[1st Match (N), Chennai, March 22, [Tushar Des...","[1st Match (N), Chennai, March 22, [Maheesh Th...","[1st Match (N), Chennai, March 22, [Mustafizur...","[1st Match (N), Chennai, March 22, [Ravindra J...","[1st Match (N), Chennai, March 22, []]","[1st Match (N), Chennai, March 22, [Mohammed S...","[1st Match (N), Chennai, March 22, [Yash Dayal...","[1st Match (N), Chennai, March 22, [Alzarri Jo...","[1st Match (N), Chennai, March 22, [Karn Sharm...","[1st Match (N), Chennai, March 22, [Mayank Dag...","[1st Match (N), Chennai, March 22, [Cameron Gr...","[1st Match (N), Chennai, March 22, [Glenn Maxw...",None,None,None
1,"[2nd Match (D/N), Mullanpur, March 23, []]","[2nd Match (D/N), Mullanpur, March 23, [Sam Cu...","[2nd Match (D/N), Mullanpur, March 23, [Arshde...","[2nd Match (D/N), Mullanpur, March 23, [Kagiso...","[2nd Match (D/N), Mullanpur, March 23, [Harpre...","[2nd Match (D/N), Mullanpur, March 23, [Rahul ...","[2nd Match (D/N), Mullanpur, March 23, [Harsha...","[2nd Match (D/N), Mullanpur, March 23, []]","[2nd Match (D/N), Mullanpur, March 23, [Khalee...","[2nd Match (D/N), Mullanpur, March 23, [Ishant...","[2nd Match (D/N), Mullanpur, March 23, [Mitche...","[2nd Match (D/N), Mullanpur, March 23, [Axar P...","[2nd Match (D/N), Mullanpur, March 23, [Kuldee...","[2nd Match (D/N), Mullanpur, March 23, [Sumit ...",None,None,None
2,"[3rd Match (N), Eden Gardens, March 23, []]","[3rd Match (N), Eden Gardens, March 23, [Bhuvn...","[3rd Match (N), Eden Gardens, March 23, [Marco...","[3rd Match (N), Eden Gardens, March 23, [T Nat...","[3rd Match (N), Eden Gardens, March 23, [Pat C...","[3rd Match (N), Eden Gardens, March 23, [Mayan...","[3rd Match (N), Eden Gardens, March 23, [Shahb...","[3rd Match (N), Eden Gardens, March 23, []]","[3rd Match (N), Eden Gardens, March 23, [Mitch...","[3rd Match (N), Eden Gardens, March 23, [Harsh...","[3rd Match (N), Eden Gardens, March 23, [Varun...","[3rd Match (N), Eden Gardens, March 23, [Sunil...","[3rd Match (N), Eden Gardens, March 23, [Andre...","[3rd Match (N), Eden Gardens, March 23, [Suyas...",None,None,None
3,"[4th Match (D/N), Jaipur, March 24, []]","[4th Match (D/N), Jaipur, March 24, [Mohsin Kh...","[4th Match (D/N), Jaipur, March 24, [Naveen-ul...","[4th Match (D/N), Jaipur, March 24, [Krunal Pa...","[4th Match (D/N), Jaipur, March 24, [Ravi Bish...","[4th Match (D/N), Jaipur, March 24, [Yash Thak...","[4th Match (D/N), Jaipur, March 24, [Ayush Bad...","[4th Match (D/N), Jaipur, March 24, []]","[4th Match (D/N), Jaipur, March 24, [Trent Bou...","[4th Match (D/N), Jaipur, March 24, [Nandre Bu...","[4th Match (D/N), Jaipur, March 24, [Ravichand...","[4th Match (D/N), Jaipur, March 24, [Avesh Kha...","[4th Match (D/N), Jaipur, March 24, [Yuzvendra...","[4th Match (D/N), Jaipur, March 24, [Sandeep S...",None,None,None
4,"[5th Match (N), Ahmedabad, March 24, []]","[5th Match (N), Ahmedabad, March 24, [Hardik P...","[5th Match (N), Ahmedabad, March 24, [Luke Woo...","[5th Match (N), Ahmedabad, March 24, [Jasprit ...","[5th Match (N), Ahmedabad, March 24, [Shams Mu...","[5th Match (N), Ahmedabad, March 24, [Piyush C...","[5th Match (N), Ahmedabad, March 24, [Naman Dh...","[5th Match (N), Ahmedabad, March 24, [Gerald C...","[5th Match (N), Ahmedabad, March 24, []]","[5th Match (N), Ahmedabad, March 24, [Azmatull...","[5th Match (N), Ahmedabad, March 24, [Umesh Ya...","[5th Match (N), Ahmedabad, March 24, [Rashid K...","[5th Match (N), Ahmedabad, March 24, [Sai Kish...","[5th Match (N), Ahmedabad, March 24, [Spencer ...","[5th Match (N), Ahmedabad, March 24, [Mohit Sh...",None,None
5,"[6th Match (N), Bengaluru, March 25, []]","[6th Match (N), Bengaluru, March 25, [Mohammed...","[6th Match (N), Bengaluru, March 25, [Yash Day...","[6th Match (N), Bengaluru, March 25, [Alzarri ...","[6th Match (N), Bengaluru, March 25, [Cameron ...","[6th Match (N), Be

In [49]:
import pandas as pd

flat_batting_data = [item for sublist in batting_data for item in sublist]
df = pd.DataFrame(flat_batting_data, columns=["MATCH_ID","a"])
a_expanded = pd.DataFrame(df['a'].tolist(), columns=["Batter", "Dismissal", "Runs", "Balls","Minutes", "4s", "6s", "SR","NA"])
result_df = pd.concat([df["MATCH_ID"], a_expanded], axis=1)


result_df = result_df.dropna(subset=['Batter'])

# Optionally, reset the index again after dropping rows
result_df.index = range(1, len(result_df) + 1)
result_df = result_df.drop('NA', axis=1)
result_df = result_df.dropna(how='all')
result_df['Dismissal'] = result_df['Dismissal'].apply(lambda x: 'not out' if x == 'not out' else 'out')
result_df.to_csv("ipl_batting_data.csv", index=False)

In [51]:
import pandas as pd
flat_bowling_data = [item for sublist in bowling_data for item in sublist]
df = pd.DataFrame(flat_bowling_data, columns=["MATCH_ID","b"])
# Expand the 'b' column into separate columns
b_expanded = pd.DataFrame(df['b'].tolist(), columns=["Bowler", "Overs", "Maidens", "Runs", "Wickets", "Economy", "0s", "4s", "6s", "Wide", "NB", "NA"])

# Concatenate with the MATCH_ID column
result_df = pd.concat([df["MATCH_ID"], b_expanded], axis=1)
result_df = result_df.dropna(subset=['Bowler'])
# Reset the index to make it clean
result_df.index = range(1, len(result_df) + 1)
result_df.to_csv("ipl_bowling_data.csv", index=False)


In [35]:
flat_match_data = [item for sublist in winner for item in sublist]
df = pd.DataFrame(flat_match_data, columns=["Match_ID","Winner"])
df = df.dropna(how='all')
# df = df.drop('NA', axis=1)
df.index = range(1, len(df) + 1)


df.to_csv("ipl_match_data.csv", index=False)